### Import Modules

In [ ]:
import os
import shutil
import random
import torch
from collections import Counter
from torch.utils.data import random_split, DataLoader
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from ultralytics import YOLO
from sklearn.metrics import ConfusionMatrixDisplay
from torch.cuda.amp import GradScaler, autocast
import time

In [ ]:
# Set seeds for reproducibility
random.seed(42)

In [ ]:
# Configuration
DATASET_DIR = "BIGDATIOTPROJ/PlantVillage"
OUTPUT_DIR = "BIGDATIOTPROJ/OutputDataset"
CLASS_BALANCE_TARGET = 2000  # Target number of samples per class
TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.2
TEST_SPLIT = 0.1
IMG_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp"}

# Augmentation Pipeline
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor()
])

In [ ]:
# Helper function to save augmented images
def save_augmented_images(class_dir, images, target_count):
    current_count = len(images)
    print(f"Balancing class: {class_dir} (current: {current_count}, target: {target_count})")

    for i in tqdm(range(current_count, target_count), desc=f"Augmenting {class_dir}"):
        original_image = random.choice(images)
        img = Image.open(original_image).convert("RGB")
        augmented_image = augmentation_transforms(img)
        augmented_img_path = os.path.join(class_dir, f"aug_{i}.jpg")
        transforms.ToPILImage()(augmented_image).save(augmented_img_path)

In [ ]:
# Step 1: Balance the Dataset
def balance_dataset(input_dir, output_dir, target_count):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    class_dirs = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]
    for class_name in class_dirs:
        class_path = os.path.join(input_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        # Copy original images
        images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.lower().endswith(tuple(IMG_EXTENSIONS))]
        for img_path in images:
            shutil.copy(img_path, output_class_path)

        # Perform augmentation if needed
        save_augmented_images(output_class_path, images, target_count)


In [ ]:
def split_dataset(dataset_dir, output_dir, train_split, val_split, test_split):
    # Allow a small tolerance for floating-point precision errors
    assert abs(train_split + val_split + test_split - 1.0) < 1e-6, "Splits must sum to 1."

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    class_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    for class_name in class_dirs:
        class_path = os.path.join(dataset_dir, class_name)
        images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.lower().endswith(tuple(IMG_EXTENSIONS))]
        random.shuffle(images)

        train_count = int(len(images) * train_split)
        val_count = int(len(images) * val_split)

        train_images = images[:train_count]
        val_images = images[train_count:train_count + val_count]
        test_images = images[train_count + val_count:]

        splits = {"train": train_images, "val": val_images, "test": test_images}
        for split, split_images in splits.items():
            split_dir = os.path.join(output_dir, split, class_name)
            os.makedirs(split_dir, exist_ok=True)
            for img_path in split_images:
                shutil.copy(img_path, split_dir)


In [ ]:
print("Balancing dataset...")
balance_dataset(DATASET_DIR, OUTPUT_DIR, CLASS_BALANCE_TARGET)

print("Splitting dataset...")
split_dataset(OUTPUT_DIR, OUTPUT_DIR, TRAIN_SPLIT, VAL_SPLIT, TEST_SPLIT)

print("Dataset prepared and saved to:", OUTPUT_DIR)

### EfficientnetB0

In [ ]:
# Configuration
DATA_DIR = "BIGDATIOTPROJ/OutputDataset/Balanced_Split_Dataset"  
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001
NUM_CLASSES = 15
dropout_p = 0.2
in_f = 1280  
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = ["Pepper__bell___Bacterial_spot", "Pepper__bell___healthy","Potato___Early_blight", "Potato___healthy", "Potato___Late_blight", "Tomato__Target_Spot", "Tomato__Tomato_mosaic_virus", "Tomato__Tomato_YellowLeaf__Curl_Virus", "Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_healthy", "Tomato_Late_blight", "Tomato_Leaf_Mold", "Tomato_Septoria_leaf_spot", "Tomato_Spider_mites_Two_spotted_spider_mite"]

# Transforms for training and validation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=train_transforms)
val_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_test_transforms)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model
def create_effnet(num_classes):
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
    # Replace the classifier head
    nn.Sequential(
        nn.Dropout(p=dropout_p, inplace=True),
        nn.Linear(in_features=in_f, out_features=num_classes, bias=True))
    return model

# Training and validation loop
def train_effnet(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience):
    train_acc_history = []
    val_acc_history = []
    train_loss_history = []
    val_loss_history = []

    best_val_loss = float("inf")
    best_model_weights = None
    patience_counter = 0

    model.to(DEVICE)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 20)

        # Training Phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels)

        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(epoch_train_loss)
        train_acc_history.append(epoch_train_acc.item())

        # Validation Phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels)

        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(epoch_val_loss)
        val_acc_history.append(epoch_val_acc.item())

        print(f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}")
        print(f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}")

        # Early Stopping Logic
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_weights = model.state_dict()
            patience_counter = 0  # Reset patience counter
            print("Validation loss improved; saving model weights.")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss for {patience_counter} epoch(s).")

        if patience_counter >= patience:
            print(f"Stopping early due to no improvement for {patience} consecutive epochs.")
            break

    # Load the best weights before returning
    if best_model_weights:
        model.load_state_dict(best_model_weights)

    return model, train_loss_history, val_loss_history, train_acc_history, val_acc_history

# Test loop
def test_effnet(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    print(f"Test Accuracy: {test_acc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    return cm

# Plot Training and Validation Metrics
def plot_training_metrics(train_loss, val_loss, train_acc, val_acc, epochs):
    plt.figure(figsize=(14, 8))

    # Plot Training and Validation Loss
    plt.subplot(2, 1, 1)
    plt.plot(range(1, epochs + 1), train_loss, label="Training Loss", color="blue")
    plt.plot(range(1, epochs + 1), val_loss, label="Validation Loss", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.grid()

    # Plot Training and Validation Accuracy
    plt.subplot(2, 1, 2)
    plt.plot(range(1, epochs + 1), train_acc, label="Training Accuracy", color="blue")
    plt.plot(range(1, epochs + 1), val_acc, label="Validation Accuracy", color="orange")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()


In [ ]:
# Define patience
PATIENCE = 3
model = create_effnet(NUM_CLASSES)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model with early stopping
print("Starting training...")
trained_model, train_loss_history, val_loss_history, train_acc_history, val_acc_history = train_effnet(
    model, train_loader, val_loader, criterion, optimizer, EPOCHS, PATIENCE
)

# Plot Metrics
epochs = len(train_loss_history)
plot_training_metrics(train_loss_history, val_loss_history, train_acc_history, val_acc_history, epochs)

# Test the model
print("Starting testing...")
cm = test_effnet(trained_model, test_loader)
plot_confusion_matrix(cm, class_names)

# Save the trained model
torch.save(trained_model.state_dict(), "efficientnet_b0_early_stopping.pth")
print("Model saved as 'efficientnet_b0_early_stopping.pth'")

### YOLO11

In [ ]:
yolomodel = YOLO('yolo11n-cls.pt')

results = yolomodel.train(data = DATA_DIR, epochs = 10, imgsz = 256, device = 'cpu')



### ResNet50

In [ ]:
# Device Configuration
DEVICE = "cpu"

# Paths
DATASET_DIR =   "OutputDataset/Balanced_Split_Dataset"
MODEL_SAVE_PATH = "BIGDATIOTPROJ"

# Hyperparameters
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 50
PATIENCE = 3

# Data Transforms
# Data Transforms
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize all images to 224x224
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),  # Ensure validation images are also resized
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),  # Ensure test images are also resized
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}


# Load Dataset
image_datasets = {x: datasets.ImageFolder(os.path.join(DATASET_DIR, x), data_transforms[x])
                  for x in ["train", "val", "test"]}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
               for x in ["train", "val", "test"]}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "val", "test"]}
class_names = image_datasets["train"].classes

# Model Definition
resmod = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_features = resmod.fc.in_features
resmod.fc = nn.Sequential(
        nn.Linear(in_features=2048, out_features=15, bias=True)
    )
resmod = resmod.to(DEVICE)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resmod.parameters(), lr=LEARNING_RATE)

# Train Function with Early Stopping
def train_resnet50(
    model, dataloaders, criterion, optimizer, num_epochs, patience
):
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []

    best_val_loss = float("inf")
    best_model_weights = None
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 20)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase.capitalize()}"):
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == "train":
                train_loss_history.append(epoch_loss)
                train_acc_history.append(epoch_acc.item())
            else:
                val_loss_history.append(epoch_loss)
                val_acc_history.append(epoch_acc.item())

                # Early Stopping
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    best_model_weights = model.state_dict()
                    patience_counter = 0
                    print("Validation loss improved; saving model weights.")
                else:
                    patience_counter += 1
                    print(f"No improvement in validation loss for {patience_counter} epoch(s).")

        print(f"Train Loss: {train_loss_history[-1]:.4f} Acc: {train_acc_history[-1]:.4f}")
        print(f"Val Loss: {val_loss_history[-1]:.4f} Acc: {val_acc_history[-1]:.4f}")

        if patience_counter >= patience:
            print(f"Stopping early due to no improvement for {patience} consecutive epochs.")
            break

    if best_model_weights:
        model.load_state_dict(best_model_weights)

    return model, train_loss_history, val_loss_history, train_acc_history, val_acc_history

# Test Function
def test_resnet(model, test_loader):
    model.eval()
    test_correct = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            test_correct += torch.sum(preds == labels)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_acc = test_correct.double() / dataset_sizes["test"]
    print(f"Test Accuracy: {test_acc:.4f}")
    return all_labels, all_preds

In [ ]:
print("Starting ResNet-50 training with early stopping...")
trained_model, train_loss_history, val_loss_history, train_acc_history, val_acc_history = train_resnet50(
    resmod, dataloaders, criterion, optimizer, EPOCHS, PATIENCE
)

print("Training completed. Plotting metrics...")
plot_training_metrics(train_loss_history, val_loss_history, train_acc_history, val_acc_history, len(train_loss_history))

print("Testing the trained model...")
test_labels, test_preds = test_resnet(trained_model, dataloaders["test"])
cm = confusion_matrix(test_labels, test_preds)
plot_confusion_matrix(cm, class_names)

torch.save(trained_model.state_dict(), MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}.")

### Mobilenet V2

In [ ]:
# Define device
DEVICE = "cpu"

# Hyperparameters
BATCH_SIZE = 32
EPOCHS = 25
LEARNING_RATE = 0.001
PATIENCE = 3  # Early stopping patience

# Data Transforms
common_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Augmentation for train set
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load Datasets
data_dir = "C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/OutputDataset/Balanced_Split_Dataset"
splits = ["train", "val", "test"]
datasets_dict = {}
dataloaders_dict = {}

# Check for each split and create datasets/dataloaders
for split in splits:
    split_path = os.path.join(data_dir, split)
    if os.path.exists(split_path) and len(os.listdir(split_path)) > 0:
        transform = train_transforms if split == "train" else common_transforms
        datasets_dict[split] = datasets.ImageFolder(split_path, transform=transform)
        dataloaders_dict[split] = DataLoader(datasets_dict[split], batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        print(f"{split.capitalize()} data loaded. Classes: {len(datasets_dict[split].classes)}")
    else:
        print(f"Warning: '{split}' directory missing or empty. Skipping...")

# Ensure at least train and val data are available
if "train" not in datasets_dict or "val" not in datasets_dict:
    raise ValueError("Both 'train' and 'val' datasets must be available!")

# Load Pretrained MobileNetV2
mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
mobilenet.classifier[1] = nn.Sequential(
        nn.Dropout(p=0.2, inplace=False),
        nn.Linear(in_features=1280, out_features=15, bias=True)
    )
mobilenet = mobilenet.to(DEVICE)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=LEARNING_RATE)

# Early Stopping Parameters
best_val_loss = float('inf')
patience_counter = 0

# Training Loop
train_loss_history, val_loss_history = [], []
train_acc_history, val_acc_history = [], []

print("Starting MobileNetV2 training...")

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    epoch_start_time = time.time()
    
    for phase in ["train", "val"]:
        if phase not in dataloaders_dict:
            continue
        
        if phase == "train":
            mobilenet.train()
        else:
            mobilenet.eval()
        
        running_loss, running_corrects = 0.0, 0
        
        with tqdm(dataloaders_dict[phase], desc=f"{phase.capitalize()} Phase", leave=False) as tbar:
            for inputs, labels in tbar:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = mobilenet(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                tbar.set_postfix(loss=loss.item())
        
        epoch_loss = running_loss / len(datasets_dict[phase])
        epoch_acc = running_corrects.double() / len(datasets_dict[phase])
        
        if phase == "train":
            train_loss_history.append(epoch_loss)
            train_acc_history.append(epoch_acc.item())
        else:
            val_loss_history.append(epoch_loss)
            val_acc_history.append(epoch_acc.item())
            
            # Early stopping check
            if epoch_loss < best_val_loss:
                best_val_loss = epoch_loss
                patience_counter = 0
                torch.save(mobilenet.state_dict(), "best_mobilenet_model.pth")
            else:
                patience_counter += 1
        
        # Print metrics after each phase
        print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    
    epoch_duration = time.time() - epoch_start_time
    remaining_time = epoch_duration * (EPOCHS - epoch - 1)
    print(f"Epoch Duration: {epoch_duration:.2f}s. Estimated Time Remaining: {remaining_time / 60:.2f} min.")
    
    if patience_counter >= PATIENCE:
        print(f"Early stopping at epoch {epoch + 1}")
        break

# Load the best model
mobilenet.load_state_dict(torch.load("best_mobilenet_model.pth"))

# Testing Phase
print("\nEvaluating on test set...")
mobilenet.eval()
test_corrects = 0
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in tqdm(dataloaders_dict["test"], desc="Testing"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = mobilenet(inputs)
        _, preds = torch.max(outputs, 1)
        
        test_corrects += torch.sum(preds == labels.data)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = test_corrects.double() / len(datasets_dict["test"])
print(f"Test Accuracy: {test_acc:.4f}")

# Plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=datasets_dict["test"].classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data from the training log
epochs = [1, 2, 3, 4, 5]
train_loss = [0.2726, 0.1045, 0.0821, 0.0720, 0.0650]
val_loss = [0.1329, 0.0465, 0.1012, 0.0558, 0.0812]
train_acc = [0.9163, 0.9655, 0.9728, 0.9767, 0.9785]
val_acc = [0.9558, 0.9845, 0.9700, 0.9808, 0.9762]

# Plot Loss values
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy values
plt.subplot(1, 2, 2)
plt.plot(epochs, train_acc, label='Train Accuracy', marker='o')
plt.plot(epochs, val_acc, label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=datasets_dict["test"].classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Class names
class_names = [
    "Pepper__bell___Bacterial_spot", "Pepper__bell___healthy", "Potato___Early_blight", 
    "Potato___healthy", "Potato___Late_blight", "Tomato__Target_Spot", 
    "Tomato__Tomato_mosaic_virus", "Tomato__Tomato_YellowLeaf__Curl_Virus", 
    "Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_healthy", 
    "Tomato_Late_blight", "Tomato_Leaf_Mold", "Tomato_Septoria_leaf_spot", 
    "Tomato_Spider_mites_Two_spotted_spider_mite"
]

# Confusion matrix data
confusion_matrix = np.array([
    [197, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 198, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 199, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 208, 0, 0, 0, 0, 0, 0, 6, 0, 0],
    [1, 0, 0, 0, 0, 0, 188, 5, 1, 4, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 2, 191, 4, 2, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 199, 1, 0, 0, 0, 0, 0],
    [4, 0, 2, 0, 0, 0, 3, 1, 0, 190, 0, 0, 0, 0, 0],
    [4, 0, 2, 0, 0, 0, 3, 1, 0, 0, 199, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 5, 0, 0, 0, 4, 3, 188, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 321, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 199, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 199]
])

# Plot the confusion matrix
plt.figure(figsize=(12, 10))
sns.set(font_scale=0.8)  # Adjust for readability
ax = sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)

# Axis labels and title
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# Rotate axis labels for better readability
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Show the plot
plt.tight_layout()
plt.show()
